In [12]:
import soundfile
import sounddevice
import codecs
import glob
import multiprocessing
import re
import nltk
import gensim.models.word2vec as w2v
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import gensim
from sklearn.model_selection import train_test_split
import pprint
from pyautogui import alert

C:\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [13]:
import keras
smodel = keras.models.load_model("./DeadSimpleSpeechRecognizer/chippy_v1.model")
model = keras.models.load_model("model1.chatbot")

C:\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
# Second dimension of the feature is dim2
feature_dim_2 = 11

# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 50
batch_size = 100
verbose = 1
num_classes = 2

In [15]:
interrupted = False
yesno_state = False
listen_state = False
talk_state = False
cmd = ""
headless = False
i_quit = False

### WORD TOKEN UTILITY

In [16]:
from chat_wordvec import *
from intent_builder import *
from chat_iface import *
import chitchat as chitchat
from preprocess import *

['affirmasi', 'cctv_lewatmana', 'goodbye', 'lihat_gambar', 'negasi', 'salam']
from data.affirmasi import affirmasi
from data.cctv_lewatmana import cctv_lewatmana
from data.goodbye import goodbye
from data.lihat_gambar import lihat_gambar
from data.negasi import negasi
from data.salam import salam


In [17]:
humanstate={
    'sequence': 0,
    'intent':'None',
    'intentscore': 0.,
    'followup': 'None',
    'intentfu': 'None',
    'context': 'None',
    'content': 'None',
    'lastmsg': 'None'
}
botstate={
    'sequence': 0,
    'intent':'None',
    'intentscore': 0.,
    'followup': 'None',
    'intentfu': 'None',
    'context': 'None',
    'content': 'None',
    'lastmsg': 'None',
    'prompt': 'None',
    'process': 'None'
}

In [18]:
def pred(sentence):
    """memprediksi masuk intent apakah sebuah kalimat."""
    predictions=model.predict(np.array([swv_ar(twl(sentence))]))
    cls_pred = np.argmax(predictions,axis=1)
   
    result={'intent': intent[cls_pred[0]], 'score': predictions.max(), 'sentence': sentence}
    return result


def sequence(sentence,humanstate,botstate):
    """urutan memproses sebuah kalimat menjadi sebuah reply"""
    print("SENTENCE:",sentence)
    humanstate,botstate = input_classifier(sentence,humanstate,botstate)
    humanstate,botstate = input_processor(humanstate,botstate)
    print("HEREBOT:",botstate)
    output,humanstate,botstate = reply_creator(humanstate,botstate)
    return output,humanstate,botstate

In [19]:
listening = False

def input_classifier(sentence,humanstate,botstate):
    result = pred(sentence)
    humanstate['lastmsg']=sentence
    humanstate['intent']=result['intent']
    humanstate['intentscore']=result['score']
    scoremasukan = result['score']
    minimum_score = min_pred[result['intent']]
    
    #PERUBAHAN INTENT APABILA SCORE DIBAWAH YANG DI TENTUKAN
    if  scoremasukan > minimum_score :
        pass
    else:
        #karena lebih kecil dari score- maka ini chitchat
        humanstate['intent']='chitchat'
    return humanstate,botstate

def input_processor(humanstate,botstate):
    global listening
    botstate['lastmsg']=botstate['prompt']
    
    if botstate['intent']=='goodbye':
        print("Goodbye")
        listening=False
        
    #EXEKUSI FOLLOWUP SECARA PRIORITAS - apakah bot sebelumnya minta followup?
    if botstate['followup'] != 'None':
            #kalau negasi setelah followup bot maka batal semua
            if humanstate['intent'] == 'negasi':
                botstate['intent']='None'
                botstate['followup']='None'
                botstate['name']='None'
                botstate['prompt']='batal'
                instruction="{'name':'None','followup':'None','intentfu':'None','prompt':'pembatalan'}"
            else:
                instruction = botstate['intentfu']+'.'+"input(\'"+humanstate['lastmsg']+"\')"
                print(instruction)
    else:
        
        if humanstate['intent'] != 'chitchat':
            instruction = humanstate['intent']+'.'+"input(\'"+humanstate['lastmsg']+"\')"
            print(instruction)
        else:
            if humanstate['intent'] == 'goodbye':
                print("I will quit")
            rd = alert(text='Realy close this app?', title='REQUEST', button='OK', timeout=5000)
            if rd == "OK":
                listening = False
                instruction='print("I QUIT")'
                try:
                    s_iface.close()
                except:
                    pass
            #TARUH DISINI UNTUK RUTIN CHITCHAT - dan kurang mengerti apa yang di bilang.
        
        print(instruction, "intent driven")
        
    #EXECUTE COMMAND =======================
    rslt = eval(instruction)
    # ======================================            
    
    
    #proses kembali jawaban dari EVAL
    if type(rslt) is not dict:
        botstate['intent'] = 'None'
        botstate['followup'] = 'None'
        botstate['prompt'] = 'oke'
        
    else:
        botstate['intent'] = humanstate['intent']
        botstate['followup'] = rslt['name']
        botstate['intentfu'] = rslt['followup']
        botstate['prompt']= rslt['prompt']

    return humanstate, botstate

def reply_creator(humanstate,botstate):
    output=botstate['prompt']
    
    if botstate['process'] != 'None':
        eval(botstate['process'])
    pprint.pformat(humanstate)
    print(pprint.pformat(botstate))
    return output, humanstate, botstate    

min_pred = {'lihat_gambar': 0.2 ,
            'cctv_lewatmana': 0.2, 
            'salam': 0.1, 
            'affirmasi': 0.1,
            'negasi': 0.1,
            'goodbye': 0.1}

In [20]:
sequence("see you",humanstate,botstate)

SENTENCE: see you
goodbye.input('see you')
goodbye.input('see you') intent driven
HEREBOT: {'sequence': 0, 'intent': 'goodbye', 'intentscore': 0.0, 'followup': 'None', 'intentfu': 'None', 'context': 'None', 'content': 'None', 'lastmsg': 'None', 'prompt': 'sampai jumpa', 'process': 'None'}
{'content': 'None',
 'context': 'None',
 'followup': 'None',
 'intent': 'goodbye',
 'intentfu': 'None',
 'intentscore': 0.0,
 'lastmsg': 'None',
 'process': 'None',
 'prompt': 'sampai jumpa',
 'sequence': 0}


('sampai jumpa',
 {'content': 'None',
  'context': 'None',
  'followup': 'None',
  'intent': 'goodbye',
  'intentfu': 'None',
  'intentscore': 0.42131236,
  'lastmsg': 'see you',
  'sequence': 0},
 {'content': 'None',
  'context': 'None',
  'followup': 'None',
  'intent': 'goodbye',
  'intentfu': 'None',
  'intentscore': 0.0,
  'lastmsg': 'None',
  'process': 'None',
  'prompt': 'sampai jumpa',
  'sequence': 0})

In [21]:
import soundfile
import itertools
import time
import librosa
import sounddevice


DURATION=20
gain=10
high=2000
low=100
screenwidth=79
predicting=False


def record(length=1, reclength=1, filename=None, thres=0):
    """ 
    Merekam suara secara stream dan metode callback
    """

    global cumulated_status, end_count, start_count, recording, magnitudo, audiodata, predicting, i_quit, listening
    predicting=False
    listening=True
    end_count=False
    start_count = 0
    recording=False
    magnitudo=[]
    audiodata=[]
    try:
        import sounddevice as sd

        #samplerate = sd.query_devices(args.device, 'input')['default_samplerate']
        samplerate = 16000.0

        delta_f = (high - low) / screenwidth
        fftsize = np.ceil(samplerate / delta_f).astype(int)
        low_bin = int(np.floor(low / delta_f))

        cumulated_status = sd.CallbackFlags()

        def callback(indata, frames, time, status):
            global cumulated_status, audiodata, magnitudo, end_count, start_count, recording, smodel, predicting, i_quit
            
            cumulated_status |= status
            if any(indata):
                magnitude = np.abs(np.fft.rfft(indata[:, 0], n=fftsize))
                magnitude *= gain / fftsize

                rms = librosa.feature.rmse(S=indata)
                rms = int(rms*32768)
                start_count += 1
                if rms>=thres:
                    if not recording :                    #and not end_count
                        #print("Start record")
                        recording = True
                        start_count = 0
                        
                        
                if recording:
                    audiodata.extend(itertools.chain(indata.tolist()))
                    magnitudo.append(magnitude)
                    if start_count == int(samplerate / (samplerate * DURATION / 1000)):
                        #print("End record")
                        start_count=0
                        end_count=True
                        recording=False
                        try:
                            if not predicting:
                                print("Predict")
                                soundfile.write("temp.wav",audiodata,16000)
                                predict("temp.wav", model=smodel)
                                predicting=False
                            pass
                        except:
                            pass
                        audiodata=[]



        with sd.InputStream(device=None, channels=1, callback=callback,
                            blocksize=int(samplerate * DURATION / 1000),
                            samplerate=samplerate):
            while True:
                #response = input()
                #if response in ('', 'q', 'Q'):
                if listening==False:
                #time.sleep(length)
                    break
            if filename!=None: soundfile.write(filename,audiodata,16000)

        if cumulated_status:
            logging.warning(str(cumulated_status))
    except Exception as e:
        print(e)

In [22]:

# Predicts one sample
def predict(filepath, model):
    
    global s_iface, humanstate, botstate,predicting
    predicting=True
    sv,sr = soundfile.read('./Ring09.wav')
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    y_pred = model.predict(sample_reshaped)
    y_max = np.max(y_pred) #nilai akurasi prediksi
    ypred = np.argmax(y_pred) #yg di prediksi adalah cipi
    cipi_recognized = (y_max>0.9 and ypred==1)
    
    if cipi_recognized:
        followup = False
        print("Predicted:" ,get_labels()[0][ypred],y_max, ypred)
        sounddevice.play(sv,samplerate=16000)
        #sudah di kenali 
        
        speech = get_speech(s_iface)
        if speech == "":
            speech = "hello"
        reply,humanstate,botstate = sequence(speech,humanstate,botstate)
        clear(s_iface)
        testalk(botstate['prompt'],s_iface)
        print(botstate['followup'])
        bot_lagi_followup = (botstate['followup'] != 'None')
        
        if bot_lagi_followup:
            followup=True
            print('followup teridentifikasi')
        
        while followup:
            playding()
            speech = get_speech(s_iface)
            reply,humanstate,botstate = sequence(speech,humanstate,botstate)
            print('sudah diambil')
            clear(s_iface)
            testalk(botstate['prompt'],s_iface)
            if botstate['followup'] != 'None':
                followup=True
                
            else:
                followup=False
                break
    predicting=False
    return get_labels()[0][ypred]

In [23]:
from selenium import webdriver

headless = False
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--use-fake-ui-for-media-stream")
if headless:
    chrome_options.add_argument("--headless")  
    
def start_iface():
    global s_iface
    s_iface = webdriver.Chrome(chrome_options=chrome_options)
    s_iface.get('https://translate.google.com/?#id/id')
    assert "Google Translate" in s_iface.title

    
def mainbrowser(url):
    global m_iface
    m_iface = webdriver.Chrome(chrome_options=chrome_options)
    m_iface.get('https://translate.google.com/?#id/id')

In [24]:
from chat_iface import *

In [25]:
start_iface()

try:
    talk("mau nonton willy wonka?", s_iface)
except Exception as e:
    del s_iface
    start_iface()
    talk("Tolong di ulang lagi", s_iface)

In [26]:
def testalk(sentence,s_iface):
    talking = True
    talk(sentence, s_iface)
    while talking:
        if not cek_talk_or_speak(s_iface):
            talking=False
            print("X")
            clear(s_iface)
        else:
            print("_",end='')
            time.sleep(0.02)

In [27]:
predict("temp.wav",model=smodel)

Predicted: cipi 0.99998975 1
SENTENCE: Sipo sipo
Problem at phrase: sipo
Problem at phrase: sipo
Goodbye
affirmasi.input('Sipo sipo')
affirmasi.input('Sipo sipo') intent driven
HEREBOT: {'sequence': 0, 'intent': 'affirmasi', 'intentscore': 0.0, 'followup': 'None', 'intentfu': 'None', 'context': 'None', 'content': 'None', 'lastmsg': 'sampai jumpa', 'prompt': 'oke', 'process': 'None'}
{'content': 'None',
 'context': 'None',
 'followup': 'None',
 'intent': 'affirmasi',
 'intentfu': 'None',
 'intentscore': 0.0,
 'lastmsg': 'sampai jumpa',
 'process': 'None',
 'prompt': 'oke',
 'sequence': 0}
X
None


'cipi'

In [ ]:
record(length=55,filename="record.wav",thres=1100)
print("END PREDICTING+++++++++++++++")

In [21]:
from pyautogui import alert

In [23]:
alert(text="Hello",timeout=4000)

'OK'

In [27]:
pyautogui.typewrite('Hello world!')